In [6]:
import json
from operator import itemgetter
import numpy as np

In [7]:
refPath = "/Users/amira/Desktop/corpus_stt/medias_corrected/fr-FR_news_0120be6c-7b6d-447a-bae2-d79a9149ddd6_Journal-Polynesie/reference_transcript_to_evaluate.json"
hypPath = "/Users/amira/Desktop/corpus_stt/medias_corrected/fr-FR_news_0120be6c-7b6d-447a-bae2-d79a9149ddd6_Journal-Polynesie/vocapia/nlp_transcript_norm.json"

In [8]:
#method to read a transcript json file
def read_transcript(transcriptPath):
    try:
        return json.load(open(transcriptPath,'rt',encoding='utf-8'))
    except FileNotFoundError:
        return None

In [9]:
#method to get the speaker time intervals
def get_speaker_screen_time(transcript):
    speakers = transcript['speakers']
    words = transcript['words']
    speakerScreenTimeList = []
    
    for speaker in speakers : 

        speakerId = speaker['id']
        speakerScreenTime =[]
        
        interList = []
        i = 0
        while i < len(words):
            interList = []
            while i < len(words) and words[i]['speakerId'] == speakerId:
                interList.append(words[i])
                i+=1
            
            if len(interList) > 0 :
                screenTime = {
                    "startTime" : interList[0]['startTime'],
                    "endTime" : interList[-1]['endTime']
                }
                speakerScreenTime.append(screenTime)
                interList = []
            else :
                i+=1

        
        speaker['speakerScreenTime'] = speakerScreenTime
        speakerScreenTimeList.append(speaker)
                    
    return speakerScreenTimeList
           

In [86]:
def get_speaker_vector(speakerId, screenTime, totalTime, silenceList):
    speakerList = [0]*(totalTime + 1)
    
    for timeSlot in screenTime:
        
        startTime = int(timeSlot["startTime"])
        endTime = int(timeSlot['endTime'])
        for i in range(startTime, endTime + 1, 1):
            speakerList[i] = 1
            silenceList[i] = 0
    
    return speakerList, silenceList
    

In [97]:
def built_speaker_arrays(speakerScreenTime, totalTime):
    listSpeakers = []
    silenceList = [1]*(totalTime + 1)
    for elem in screenTime:
        speakerList, silenceList = get_speaker_vector(elem['id'], elem["speakerScreenTime"], totalTime, silenceList)
        listSpeakers.append(speakerList)

    listSpeakers = [silenceList] + listSpeakers
    return np.array(listSpeakers)


In [87]:
hypTranscript = read_transcript(hypPath)
refTranscript = read_transcript(refPath)

In [88]:
hypSpeakerScreenTime = get_speaker_screen_time(hypTranscript)
refSpeakerScreenTime = get_speaker_screen_time(refTranscript)

In [89]:
totalRefTime =  int(refTranscript['words'][-1]['endTime'])
totalHypTime =  int(hypTranscript['words'][-1]['endTime'])

In [98]:
hypRes = built_speaker_arrays(hypSpeakerScreenTime, totalHypTime)
refRes = built_speaker_arrays(refSpeakerScreenTime, totalRefTime)

NameError: name 'screenTime' is not defined

[[1 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]]
